# Web Scrapping

In [1]:
import time

import bs4
import requests
import pandas as pd

In [2]:
# Creates a variable
agent = "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/113.0.0.0 Safari/537.36"

In [3]:
# Sets headers with request agent
headers= { "User-Agent": agent }

In [4]:
def consult(url):
    # Try to execute requests
    try:
        req = requests.get(url, headers = headers, timeout = 1) 
        return req.content      
    except:
        pass


def capture_html(url):
    html = consult(url)
    soup = bs4.BeautifulSoup(html, "html.parser")
    return soup


def get_html_header(html):
    dt = html.find_all("dt")
    dd = html.find_all("dd")
    values = {}
    for key, val in zip(dt, dd):
        values[key.get_text()] = val.get_text()

    return values


def get_proposition(proposition_id, year):
    # Sets the address
    url = f"https://www.legislador.com.br//LegisladorWEB.ASP?WCI=ProposicaoTexto&ID=3&TPProposicao=1&nrProposicao={proposition_id}&aaProposicao={year}"
    html = capture_html(url)
    values = get_html_header(html)
    values["Proposição"] = proposition_id
    values["Ano"] = year
    values["Texto"] = html.p.get_text() # Page has a single <p> tag

    return values

# Connection to the Database

In [5]:
import pyodbc

In [6]:
def sql_select(query):
    # Using SSMS with Windows' authentication
    conn = pyodbc.connect("Trusted_Connection=yes",
                        driver="{ODBC Driver 17 for SQL Server}",
                        server="localhost",
                        database="Indaial")

    res = pd.read_sql_query(query, conn)
    return res

def sql_truncate(table_name):
    conn = pyodbc.connect("Trusted_Connection=yes",
                        driver="{ODBC Driver 17 for SQL Server}",
                        server="localhost",
                        database="Indaial")

    cursor = conn.cursor()
    cursor.execute(f"""
            TRUNCATE TABLE {table_name}
        """)

    conn.commit()
    cursor.close()

def rename_columns_to_db(dataframe):
    renamed = dataframe.rename(columns={
                                        "Proposição": "proposition",
                                        "Ano": "year", "Texto": "text",
                                        "Reunião": "date_meeting",
                                        "Deliberação": "date_deliberation",
                                        "Situação": "situation", "Assunto": "subject",
                                        "Autor": "author"
                                    })
    
    return renamed

def insert_propositions(dataframe):
    dataframe = rename_columns_to_db(dataframe)
    conn_df = pd.DataFrame(columns=["date_meeting", "date_deliberation", "situation", "subject", "author", "proposition", "year", "text"])
    conn_df = conn_df.append(dataframe).fillna("")
    conn_df["date_meeting"] = pd.to_datetime(conn_df["date_meeting"], format="%d/%m/%Y")
    conn_df["date_deliberation"] = pd.to_datetime(conn_df["date_deliberation"], format="%d/%m/%Y")

    conn = pyodbc.connect("Trusted_Connection=yes",
                        driver="{ODBC Driver 17 for SQL Server}",
                        server="localhost",
                        database="Indaial")

    cursor = conn.cursor()

    for _, row in conn_df.iterrows():
        cursor.execute("""INSERT INTO Propositions (
            date_meeting, date_deliberation, situation, subject, author, proposition, year, text
        ) VALUES(?, ?, ?, ?, ?, ?, ?, ?)""",
            row["date_meeting"], row["date_deliberation"], row["situation"], row["subject"], row["author"], row["proposition"], row["year"], row["text"]
        )

    conn.commit()
    cursor.close()

def insert_next_proposition(year):
    # Gets the last proposition
    year_data = sql_select(f"select proposition = max(proposition) from Propositions where year = {year}")
    last_proposition = year_data["proposition"].loc[0]

    # Verifies if a proposition already exists in current year
    if last_proposition == None:
        next_proposition = 1
    else:
        next_proposition = int(last_proposition) + 1 

    # Captures and inserts on table
    data = get_proposition(next_proposition, year)
    table = pd.DataFrame([data])
    insert_propositions(table)


def insert_year(year, max_error = 20, wait_secs = 0.5):
    errors_count = 0

    # When no proposition is found, error will start increasing
    while errors_count < max_error:
        try:
            insert_next_proposition(year)
        except:
            errors_count += 1
            pass

        time.sleep(wait_secs)


def insert_db(start_year, end_year):
    for i in range(start_year, end_year+1):
        print(f"Inserting data from {i}")
        try:
            insert_year(i)
        except:
            pass

Finished inserting data


In [36]:
#sql_truncate("Propositions")
insert_db(1996, 2023)

Inserting data from 2023
